#01. QC Example script
mainly based on https://www.sc-best-practices.org/ https://www.nature.com/articles/s41576-023-00586-w

In [ ]:
import scanpy as sc, numpy as np, pandas as pd
import seaborn as sns
from scipy.stats import median_abs_deviation
import scvi

In [ ]:
scvi.settings.seed = 0

In [ ]:
import torch
print(torch.cuda.is_available())

In [ ]:
from matplotlib.pyplot import rc_context
from matplotlib import pyplot as plt

In [ ]:
import os
os.environ["R_HOME"] = "/data/User/revolvefire/miniforge3/envs/rpy_base_240331/lib/R"
os.environ['LD_LIBRARY_PATH'] = "/data/User/revolvefire/miniforge3/envs/rpy_base_240331/lib/R/bin"

In [ ]:
import anndata2ri
import logging

In [ ]:
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro
print(ro.r(".libPaths()"))

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [ ]:
from time import gmtime, strftime
strftime("%Y-%m-%d %H:%M:%S", gmtime())

In [ ]:
mainDir = "/mnt/gmi-l1/_90.User_Data/revolvefire/fastq/PPP_GMI"

In [ ]:
objectDir = "/mnt/gmi-l1/_90.User_Data/revolvefire/projects/3.PPP_231118/231128_PPP/scobject/"

In [ ]:
file_list = ["PPP_L", "PPP_NL" ] 

In [ ]:
data_list = {}

In [ ]:
for i in file_list:
    file_path = f"{mainDir}/{i}/outs/per_sample_outs/{i}/count/sample_filtered_feature_bc_matrix.h5"
    data = sc.read_10x_h5(filename=file_path)
    data_list[i] = data

In [ ]:
data_list["PPP_L"]

In [ ]:
PPP_L = data_list["PPP_L"]
PPP_NL= data_list["PPP_NL"]


In [ ]:
adata = PPP_L
adata

In [ ]:
adata.var_names_make_unique()
adata

In [ ]:
#for human

In [ ]:
# mitochondrial genes
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes.
adata.var["hb"] = adata.var_names.str.contains(("^HB[^(P)]"))

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, percent_top=[20], log1p=True
)
adata

In [ ]:
p1 = sns.displot(adata.obs["total_counts"], bins=100, kde=False)
# sc.pl.violin(adata, 'total_counts')
p2 = sc.pl.violin(adata, ["pct_counts_mt", "pct_counts_ribo", "pct_counts_hb"], multi_panel=False)
p3 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

In [ ]:
def is_outlier(adata, metric: str, nmads: int):
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

In [ ]:
adata.obs["outlier"] = (
    is_outlier(adata, "log1p_total_counts", 5)
    | is_outlier(adata, "log1p_n_genes_by_counts", 5)
    | is_outlier(adata, "pct_counts_in_top_20_genes", 5)
)
adata.obs.outlier.value_counts()

In [ ]:
adata.obs["mt_outlier"] = is_outlier(adata, "pct_counts_mt", 3) | (
    adata.obs["pct_counts_mt"] > 8
)
adata.obs.mt_outlier.value_counts()

In [ ]:
print(f"Total number of cells: {adata.n_obs}")
adata = adata[(~adata.obs.outlier) & (~adata.obs.mt_outlier)].copy()

print(f"Number of cells after filtering of low quality cells: {adata.n_obs}")

In [ ]:
p1 = sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

In [ ]:
%%R
library(SoupX)

In [ ]:
adata_pp = adata.copy()
sc.pp.normalize_per_cell(adata_pp)
sc.pp.log1p(adata_pp)

In [ ]:
sc.pp.pca(adata_pp)
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added="soupx_groups")

# Preprocess variables for SoupX
soupx_groups = adata_pp.obs["soupx_groups"]

In [ ]:
del adata_pp

In [ ]:
cells = adata.obs_names
genes = adata.var_names
data = adata.X.T

In [ ]:
adata_raw = sc.read_10x_h5(
    filename=mainDir + "/PPP_L/outs/multi/count/raw_feature_bc_matrix.h5",
)
adata_raw.var_names_make_unique()
data_tod = adata_raw.X.T

In [ ]:
del adata_raw

In [ ]:
%%R -i data -i data_tod -i genes -i cells -i soupx_groups -o out 

# specify row and column names of data
rownames(data) = genes
colnames(data) = cells
# ensure correct sparse format for table of counts and table of droplets
data <- as(data, "sparseMatrix")
data_tod <- as(data_tod, "sparseMatrix")

# Generate SoupChannel Object for SoupX 
sc = SoupChannel(data_tod, data, calcSoupProfile = FALSE)

# Add extra meta data to the SoupChannel object
soupProf = data.frame(row.names = rownames(data), est = rowSums(data)/sum(data), counts = rowSums(data))
sc = setSoupProfile(sc, soupProf)
# Set cluster information in SoupChannel
sc = setClusters(sc, soupx_groups)

# Estimate contamination fraction
sc = setContaminationFraction(sc, 0.2)
# Infer corrected table of counts and rount to integer
out = adjustCounts(sc, roundToInt = TRUE)

In [ ]:
adata.layers["counts"] = adata.X
adata.layers["soupX_counts"] = out.T
adata.X = adata.layers["soupX_counts"]

In [ ]:
%%R
library(Seurat)
library(scater)
library(scDblFinder)
library(BiocParallel)

In [ ]:
data_mat = adata.X.T

In [ ]:
%%R -i data_mat -o doublet_score -o doublet_class

set.seed(123)
sce = scDblFinder(
    SingleCellExperiment(
        list(counts=data_mat),
    ) 
)
doublet_score = sce$scDblFinder.score
doublet_class = sce$scDblFinder.class

In [ ]:
adata.obs["scDblFinder_score"] = doublet_score
adata.obs["scDblFinder_class"] = doublet_class
adata.obs.scDblFinder_class.value_counts()